In [31]:
from langgraph.graph import StateGraph ,START,state,MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode ,tools_condition
from langchain_core.messages import AIMessage,HumanMessage
from typing_extensions import Annotated,TypedDict
from utlis.model_loader import ModelLoader
from toolkit.tools import *





In [32]:
class State(TypedDict):
    message:Annotated[list,add_messages]

In [33]:
class GraphBuilder:
    def __init__(self):
        self.model_loader=ModelLoader()
        self.llm = self.model_loader.load_llm()
        self.tools = [retriever_tool, financials_tool, tavilytool]
        llm_with_tools = self.llm.bind_tools(tools=self.tools)
        self.llm_with_tools = llm_with_tools
        self.graph = None
    
    def _chatbot_node(self,state:dict):
         return {"messages": [self.llm_with_tools.invoke(state["messages"])]}

    def build(self):
        graph_builder = StateGraph(MessagesState)
        
        graph_builder.add_node("chatbot", self._chatbot_node)
        
        tool_node=ToolNode(tools=self.tools)
        graph_builder.add_node("tools", tool_node)
        
        graph_builder.add_conditional_edges("chatbot", tools_condition)
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_edge(START, "chatbot")
        
        self.graph = graph_builder.compile()
        
    def get_graph(self):
        if self.graph is None:
            raise ValueError("Graph not built. Call build() first.")
        return self.graph 
    
    

In [34]:
graph_service = GraphBuilder()

LLM loading...


In [35]:
graph_service.build()
graph=graph_service.get_graph()

In [38]:
messages={"messages":"what iscurrent stock price of adani green?"}

In [39]:
result=graph.invoke(messages)

In [40]:
print(result)

{'messages': [HumanMessage(content='what iscurrent stock price of adani green?', additional_kwargs={}, response_metadata={}, id='59a2c3a7-2a81-4aeb-bf7e-90377560337e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'zj4xt503w', 'function': {'arguments': '{"query":"ADANIGREEN.NS"}', 'name': 'polygon_financials'}, 'type': 'function'}, {'id': 'mnx9djbj8', 'function': {'arguments': '{"query":"ADANIGREEN.NS"}', 'name': 'polygon_financials'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 1892, 'total_tokens': 1935, 'completion_time': 0.081253452, 'completion_tokens_details': None, 'prompt_time': 0.182152866, 'prompt_tokens_details': None, 'queue_time': 0.050242093, 'total_time': 0.263406318}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bea37-5370-7840-b15d-cd32027cf265-0'